In [1]:
import netCDF4
import operator
import xarray as xr
import dask
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as patches
from matplotlib import ticker
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import scipy
from scipy import signal
import math

#added from Lutsko Github
import spectrum 

In [3]:
#Good_July.nc
path_to_file = '/fast/gmooers/Real_Geography_Manuscript/Models/CAM_July.nc'
real_ds = xr.open_dataset(path_to_file)


print('files imported')

files imported


In [4]:
#heat_real_ds = real_ds.PTEQ[:, :].values
moist_real_ds = real_ds.PTEQ.values
heat_real_ds = real_ds.PTTEND.values

times = real_ds.time.values

lats = real_ds.lat.values

lons = real_ds.lon.values

In [5]:
x = 144
y = 96
z = 30
t = int(len(heat_real_ds)/(x*y))

In [6]:
spacing = 1/96

In [7]:
others = netCDF4.Dataset("/fast/gmooers/Raw_Data/extras/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-01-72000.nc")
plev = np.array(others.variables['lev'])
ps = np.array(others.variables['PS'])
g = 9.81 #m/s^2
L = 2256000.0
cp = 1004.0
#print(plev)
hyai = np.array(others.variables['hyai'])
hybi = np.array(others.variables['hybi'])
PS = 1e5
P0 = 1e5
P = P0*hyai+PS*hybi # Total pressure [Pa]
dp = P[1:]-P[:-1] # Differential pressure [Pa]
#convert from k/s to w/m^2
heat_pressure_weighted_targets = heat_real_ds*dp[None, :,None, None]*cp/g
moist_pressure_weighted_targets = moist_real_ds*dp[None, :,None, None]*L/g
print('made it')

made it


In [15]:
lon_spacing = np.abs((lons[1]-lons[0])*40075.*np.cos(lats[0])/360.0)
lat_spacing = 111.32*(lats[1]-lats[0])

In [16]:
def spectrum_gen(h, dt):
    nt = len(h)
    #Get half the length of the time series to avoid redudant information
    npositive = nt//2
    pslice = slice(1, npositive)
    #Get frequencies
    freqs = np.fft.fftfreq(nt, d=dt)[pslice] 
    #perform the fft 
    ft = np.fft.fft(h)[pslice]
    #remove imaginary componant of the fft and square
    psraw = np.conjugate(ft) *ft
    #double to account for the negative half that was removed above
    psraw *= 2.0
    #Normalization for the power spectrum
    psraw /= nt**2
    #Go fro mthe Power Spectrum to Power Density
    psdraw = psraw * dt * nt
    return freqs, psraw, psdraw
    
#spectrum_gen(s1, t[1]-t[0])

In [17]:
heat_pressure_weighted_targets.shape

(2976, 30, 96, 144)

In [19]:
def spectrum_generator_X(targets, levels, latitude_count, longitude_count, time_space):
    time_count = len(targets[:,0,0,0])
    targ_freqs, targ_psraw, targ_psdraw = spectrum_gen(np.squeeze(targets[0,0,0,:]), time_space)
    depth = len(targ_psdraw)
    target_collector = np.zeros(shape=(depth, levels, latitude_count, time_count))
    target_collector[:,:,:,:] = np.nan
    counter = 0
    for i in range(levels):
        print(i)
        for j in range(latitude_count):
            for k in range(time_count):
                target = np.squeeze(targets[k,i,j,:])
                targ_freqs, targ_psraw, targ_psdraw = spectrum_gen(target, time_space)
                target_collector[:,i,j,k] = targ_psdraw
        
    return targ_freqs, target_collector

x_heat_targ_freqs,  x_heat_target_collector = spectrum_generator_X(heat_pressure_weighted_targets, z, y, x, lon_spacing)
x_moist_targ_freqs,  x_moist_target_collector = spectrum_generator_X(moist_pressure_weighted_targets, z, y, x, lon_spacing)

0


/export/home/gmooers/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  


1


KeyboardInterrupt: 

In [20]:
def spectrum_generator_Y(targets, levels, latitude_count, longitude_count, time_space):
    time_count = len(targets[:,0,0,0])
    targ_freqs, targ_psraw, targ_psdraw = spectrum_gen(np.squeeze(targets[0,0,:,0]), time_space)
    depth = len(targ_psdraw)
    target_collector = np.zeros(shape=(depth, levels, time_count, longitude_count))
    target_collector[:,:,:,:] = np.nan
    counter = 0
    for i in range(levels):
        print(i)
        for j in range(time_count):
            for k in range(longitude_count):
                target = np.squeeze(targets[j,i,:,k])
                targ_freqs, targ_psraw, targ_psdraw = spectrum_gen(target, time_space)
                target_collector[:,i,j,k] = targ_psdraw
        
    return targ_freqs, target_collector

#y_heat_targ_freqs,  y_heat_target_collector = spectrum_generator_Y(heat_pressure_weighted_targets, z, y, x, lat_spacing)
#y_moist_targ_freqs,  y_moist_target_collector = spectrum_generator_Y(moist_pressure_weighted_targets, z, y, x, lat_spacing)

0


/export/home/gmooers/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  


1


KeyboardInterrupt: 

In [ ]:
def plotter(target_array, frequency, title_name):
    plt.plot(1/frequency, target_array, 'r', alpha = 0.5, label = "CAM5")
    plt.legend()
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('Period (Days)')
    h = plt.ylabel(r'$\frac{w^2}{m^4*day}$', fontsize = 15)
    h.set_rotation(0)
    plt.title("Power Spectrum Density for "+title_name)

In [ ]:
lat_list = lats
lon_list = lons

#method 1 - Hovmueller Github 
a = 6.37e6 #radius of the earth
dlat = np.abs(lat_list[1]-lat_list[0])
dlon = np.abs(lon_list[0]-lon_list[1])
gridCellWidth = (dlon*np.pi/180.)*a*np.cos(lat_list*np.pi/180.)

x_heat_lat_weighted_targs = x_heat_target_collector*gridCellWidth[None,None,:,None]/np.nanmean(gridCellWidth)
x_heat_lat_weighted_targs = np.nanmean(x_heat_lat_weighted_targs, axis = 2)

x_moist_lat_weighted_targs = x_moist_target_collector*gridCellWidth[None,None,:,None]/np.nanmean(gridCellWidth)
x_moist_lat_weighted_targs = np.nanmean(x_moist_lat_weighted_targs, axis = 2)

x_heat_pressure_weighted_target_output = np.nanmean(x_heat_lat_weighted_targs , axis = 1)
x_moist_pressure_weighted_target_output = np.nanmean(x_moist_lat_weighted_targs, axis = 1)

x_heat_target_avg = np.nanmean(x_heat_pressure_weighted_target_output, axis = 1)
x_moist_target_avg = np.nanmean(x_moist_pressure_weighted_target_output, axis = 1)

In [ ]:
#y_heat_lat_weighted_targs = y_heat_target_collector
#y_heat_lat_weighted_targs = np.nanmean(y_heat_lat_weighted_targs, axis = 1)

#y_moist_lat_weighted_targs = y_moist_target_collector
#y_moist_lat_weighted_targs = np.nanmean(y_moist_lat_weighted_targs, axis = 1)

#y_heat_pressure_weighted_target_output = np.nanmean(y_heat_lat_weighted_targs , axis = 1)
#y_moist_pressure_weighted_target_output = np.nanmean(y_moist_lat_weighted_targs, axis = 1)

#y_heat_target_avg = np.nanmean(y_heat_pressure_weighted_target_output, axis = 1)
#y_moist_target_avg = np.nanmean(y_moist_pressure_weighted_target_output, axis = 1)

In [ ]:
title = 'All locations'
#plotter(heat_target_avg, heat_targ_freqs, title)
#plotter(moist_target_avg, moist_targ_freqs, title)

In [ ]:
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Tropical_Spatial_CAM_Spectra_Moisture_X.npy",x_moist_target_avg)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Tropical_Spatial_CAM_Spectra_Heat_X.npy",x_heat_target_avg)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Tropical_Spatial_CAM_Spectra_freqs_Moisture_X.npy", x_moist_targ_freqs)
np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Tropical_Spatial_CAM_Spectra_freqs_Heat_X.npy", x_heat_targ_freqs)

In [ ]:
#np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Spatial_CAM_Spectra_Moisture_Y.npy", y_moist_target_avg)
#np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Spatial_CAM_Spectra_Heat_Y.npy", y_heat_target_avg)
#np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Spatial_CAM_Spectra_freqs_Moisture_Y.npy", y_moist_targ_freqs)
#np.save("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/Spatial_CAM_Spectra_freqs_Heat_Y.npy", y_heat_targ_freqs)